# Pre-trained models for text-to-speech

ASR (otomatik konuşma tanıma) ve ses sınıflandırma görevleriyle karşılaştırıldığında, önceden eğitilmiş model kontrol noktalarının sayısı çok daha azdır. Hub'da 300'e yakın uygun kontrol noktası bulacaksınız. Bu önceden eğitilmiş modeller arasında, Transformers kütüphanesinde sizin için hazır olan iki mimariye odaklanacağız - SpeechT5 ve Massive Multilingual Speech (MMS). Bu bölümde, bu önceden eğitilmiş modellerin TTS için Transformers kütüphanesinde nasıl kullanılacağını inceleyeceğiz.

## SpeechT5

SpeechT5, Junyi Ao ve arkadaşları tarafından Microsoft'tan yayınlanan ve bir dizi konuşma görevini yerine getirebilen bir modeldir. Bu ünitede, metinden konuşmaya yönüne odaklansak da, bu model metinden konuşmaya (otomatik konuşma tanıma veya konuşmacı tanımlama) ve konuşmadan konuşmaya (örneğin konuşma geliştirme veya farklı sesler arasında dönüştürme) görevlere uyarlanabilir. Bunun nedeni modelin nasıl tasarlandığı ve önceden eğitildiğidir.

SpeechT5'in merkezinde normal bir Transformer kodlayıcı-kod çözücü modeli yer alır. Tıpkı diğer Transformer'lar gibi, kodlayıcı-kod çözücü ağı da gizli temsilleri kullanarak diziden diziye dönüşümü modeller. Bu Transformatör omurgası SpeechT5'in desteklediği tüm görevler için aynıdır.

Bu Dönüştürücü, altı adet kipe özgü (konuşma/metin) ön ağ ve son ağ ile tamamlanmaktadır. Giriş konuşması veya metni (göreve bağlı olarak), Transformer'ın kullanabileceği gizli gösterimleri elde etmek için ilgili bir ön ağ aracılığıyla ön işleme tabi tutulur. Transformer'ın çıktısı daha sonra hedef modalitedeki çıktıyı oluşturmak için kullanacak olan bir post-net'e aktarılır.

Mimari bu şekilde görünmektedir (resim orijinal makaleden alınmıştır):

![](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/speecht5/architecture.jpg)

SpeechT5, farklı modalitelerin birleşik bir temsilini elde etmek için ilk olarak büyük ölçekli etiketsiz konuşma ve metin verileri kullanılarak ön eğitime tabi tutulur. Ön eğitim aşamasında tüm ön ağlar ve son ağlar aynı anda kullanılır.

Ön eğitimden sonra, tüm kodlayıcı-kod çözücü omurgası her bir görev için ince ayarlanır. Bu adımda, yalnızca belirli görevle ilgili ön ağlar ve son ağlar kullanılır. Örneğin, SpeechT5'i metinden konuşmaya kullanmak için, metin girdileri için metin kodlayıcı ön ağına ve konuşma çıktıları için konuşma kod çözücü ön ve son ağlarına ihtiyacınız olacaktır.

>>Konuşma verisi üzerinde çalışırken, konuşma sinyali spektrogramı üzerinden bazı bölgeler maskelenir. Model, bu maskelenmiş bölgeleri tahmin etmeye çalışır.

Bu yaklaşım, etiketsiz veriler üzerinde yapılan ilk ön eğitimden yararlanan farklı konuşma görevleri için ince ayarlanmış birkaç model elde edilmesini sağlar.

> İnce ayarlı modeller, ortak ön eğitimli modelden aynı ağırlık kümesini kullanarak başlasa da, nihai sürümlerin hepsi sonunda oldukça farklıdır. Örneğin, ince ayarlı bir ASR modelini alıp çalışan bir TTS modeli elde etmek için ön ağları ve son ağları değiştiremezsiniz. SpeechT5 esnektir, ancak o kadar da esnek değildir ;)

SpeechT5'in özellikle TTS görevi için kullandığı ön ve son ağların neler olduğunu görelim:

- Metin kodlayıcı ön ağı: Metin belirteçlerini kodlayıcının beklediği gizli temsillerle eşleştiren bir metin gömme katmanı. Bu, BERT gibi bir NLP modelinde olana benzer.
- Konuşma kod çözücü ön ağı: Bu, girdi olarak bir log mel spektrogramı alır ve spektrogramı gizli temsillere sıkıştırmak için bir dizi doğrusal katman kullanır.
- Konuşma kod çözücü post-net: Bu, çıktı spektrogramına eklenecek bir kalıntıyı tahmin eder ve sonuçları iyileştirmek için kullanılır.

Birleştirildiğinde, metinden konuşmaya yönelik SpeechT5 mimarisi bu şekilde görünür:

![](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/speecht5/tts.jpg)

Gördüğünüz gibi, çıktı bir log mel spektrogramıdır ve nihai bir dalga formu değildir. Hatırlarsanız, Ünite 3'te bu konuya kısaca değinmiştik. Ses üreten modellerin, vocoder olarak bilinen ek bir sinir ağı ile dalga formuna dönüştürülmesi gereken bir log mel spektrogramı üretmesi yaygındır.

Bunu nasıl yapabileceğinizi görelim.

İlk olarak, tokenizasyon ve özellik çıkarma için kullanılan işlemci nesnesi ile birlikte Hub'dan ince ayarlı TTS SpeechT5 modelini yükleyelim:

In [1]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")

preprocessor_config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/585M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [2]:
inputs = processor(
    text="Don't count the days, make the days count.", return_tensors="pt"
)

SpeechT5 TTS modeli **tek bir konuşmacı için konuşma oluşturmakla sınırlı değildir**. Bunun yerine, **belirli bir konuşmacının ses özelliklerini yakalayan sözde konuşmacı gömülerini kullanır**.

Konuşmacı gömüleri, bir konuşmacının kimliğini, ifadenin uzunluğundan bağımsız olarak sabit boyutta bir vektör olarak kompakt bir şekilde temsil etme yöntemidir. Bu göüler bir konuşmacının sesi, aksanı, tonlaması ve bir konuşmacıyı diğerinden ayıran diğer benzersiz özellikleri hakkında temel bilgileri yakalar. Bu tür gömüler konuşmacı doğrulama, konuşmacı günlüğü oluşturma, konuşmacı tanımlama ve daha fazlası için kullanılabilir. Konuşmacı gömüleri oluşturmak için en yaygın teknikler şunlardır:

- I-Vektörleri (kimlik vektörleri): I-Vektörleri Gauss karışım modeline (GMM) dayanır. Konuşmacıları, konuşmacıya özgü bir GMM'nin istatistiklerinden türetilen düşük boyutlu sabit uzunluklu vektörler olarak temsil ederler ve denetimsiz bir şekilde elde edilirler.
- **X-Vektörleri**: X-Vektörleri, derin sinir ağları (DNN'ler) kullanılarak türetilir ve zamansal bağlamı dahil ederek çerçeve düzeyinde konuşmacı bilgilerini yakalar.

**X-Vektörleri**, I-Vektörlerine kıyasla değerlendirme veri kümelerinde üstün performans gösteren son teknoloji ürünü bir yöntemdir. X-Vektörleri elde etmek için derin sinir ağı kullanılır: konuşmacılar arasında ayrım yapmak için eğitilir ve değişken uzunluktaki ifadeleri sabit boyutlu katıştırmalara eşler. Ayrıca, belirli bir konuşmacının konuşma özelliklerini kapsayacak şekilde önceden hesaplanmış bir X-Vektör konuşmacı katıştırması da yükleyebilirsiniz.

In [3]:
import torch
from datasets import load_dataset

embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
embeddings_dataset

Generating validation split:   0%|          | 0/7931 [00:00<?, ? examples/s]

Dataset({
    features: ['filename', 'xvector'],
    num_rows: 7931
})

In [4]:
speaker_embeddings = torch.tensor(embeddings_dataset[19]["xvector"]).unsqueeze(0)
speaker_embeddings.shape

torch.Size([1, 512])

In [5]:
spectrogram = model.generate_speech(inputs["input_ids"], speaker_embeddings)

Bu, log mel spektrogramı içeren (140, 80) şeklinde bir tensör çıktısı verir. İlk boyut dizi uzunluğudur ve konuşma kod çözücü ön ağı her zaman giriş dizisine dropout uyguladığı için çalıştırmalar arasında değişebilir. Bu, üretilen konuşmaya biraz rastgele değişkenlik katar.

Ancak, konuşma dalga formu oluşturmak istiyorsak, spektrogramdan dalga formuna dönüşüm için kullanılacak bir vocoder belirlememiz gerekir. Teorik olarak, 80-bin mel spektrogramlar üzerinde çalışan herhangi bir vocoder kullanabilirsiniz. Uygun bir şekilde, Transformers HiFi-GAN tabanlı bir vocoder sunmaktadır. Ağırlıkları SpeechT5'in orijinal yazarları tarafından nazikçe sağlanmıştır.

>HiFi-GAN, yüksek doğrulukta konuşma sentezi için tasarlanmış son teknoloji ürünü bir üretken düşman ağıdır (GAN). Spektrogram girdilerinden yüksek kaliteli ve gerçekçi ses dalga formları üretebilmektedir.
>
>Yüksek seviyede, HiFi-GAN bir generator ve iki diskriminatörden oluşur. Üreteç, mel-spektrogramı girdi olarak alan ve ham ses dalga formları üretmeyi öğrenen tamamen konvolüsyonel bir sinir ağıdır. Diskriminatörlerin rolü ise gerçek ve üretilmiş ses arasında ayrım yapmaktır. İki diskriminatör sesin farklı yönlerine odaklanır.
>
>HiFi-GAN, yüksek kaliteli ses kayıtlarından oluşan büyük bir veri kümesi üzerinde eğitilmiştir. Generator ve discriminator ağlarının birbirlerine karşı rekabet ettiği adversarial eğitim olarak adlandırılan bir yöntem kullanır. Başlangıçta, generator düşük kaliteli ses üretir ve diskriminatör bunu gerçek sesten kolayca ayırt edebilir. Eğitim ilerledikçe, generator diskriminatörü kandırmayı amaçlayarak çıktısını geliştirir. Diskriminatör de gerçek ses ile üretilen sesi ayırt etmede daha hassas hale gelir. Bu karşıt geri bildirim döngüsü, her iki ağın da zaman içinde gelişmesine yardımcı olur. Nihayetinde HiFi-GAN, eğitim verilerinin özelliklerine yakından benzeyen yüksek kaliteli ses üretmeyi öğrenir.

In [6]:
from transformers import SpeechT5HifiGan

vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/50.7M [00:00<?, ?B/s]

In [7]:
speech = model.generate_speech(
    inputs["input_ids"], speaker_embeddings, vocoder=vocoder
)

In [8]:
from IPython.display import Audio

Audio(speech, rate=16000)

## Bark

Bark, Suno AI tarafından suno-ai/bark'ta önerilen dönüştürücü tabanlı bir metinden konuşmaya modelidir.

SpeechT5'in aksine Bark, ham konuşma dalga formlarını doğrudan üreterek çıkarım sırasında ayrı bir ses kodlayıcıya olan ihtiyacı ortadan kaldırır - zaten entegre edilmiştir. Bu verimlilik, hem bir kodek hem de bir sıkıştırma aracı olarak hizmet veren Encodec'in kullanılmasıyla elde edilir.

Encodec ile, bellek kullanımını azaltmak için sesi hafif bir formatta sıkıştırabilir ve daha sonra orijinal sesi geri yüklemek için sıkıştırmayı açabilirsiniz. Bu sıkıştırma işlemi, her biri tamsayı vektörlerden oluşan 8 kod kitabı tarafından kolaylaştırılır. Bu kod kitaplarını sesin tamsayı biçimindeki gösterimleri veya gömülmeleri olarak düşünün. Birbirini izleyen her bir kod kitabının, önceki kod kitaplarından elde edilen ses yeniden yapılandırmasının kalitesini artırdığına dikkat etmek önemlidir. Kod kitapları tamsayı vektörleri olduğundan, bu görevde çok verimli olan dönüştürücü modeller tarafından öğrenilebilirler. Bark bu iş için özel olarak eğitilmiştir.

Daha spesifik olmak gerekirse, Bark 4 ana modelden oluşur:

- BarkSemanticModel ('metin' modeli olarak da adlandırılır): girdi olarak tokenize edilmiş metni alan ve metnin anlamını yakalayan semantik metin tokenlerini tahmin eden nedensel bir otomatik regresif dönüştürücü model. 

- BarkCoarseModel ('kaba akustik' modeli olarak da adlandırılır): BarkSemanticModel modelinin sonuçlarını girdi olarak alan nedensel bir otoregresif dönüştürücü. EnCodec için gerekli olan ilk iki ses kod kitabını tahmin etmeyi amaçlar. 

- BarkFineModel ('ince akustik' modeli), bu kez önceki kod kitaplarının gömülmelerinin toplamına dayalı olarak son kod kitaplarını yinelemeli olarak tahmin eden nedensel olmayan bir otomatik kodlayıcı dönüştürücüdür. 

- EncodecModel'den tüm kod kitabı kanallarını tahmin eden Bark, bunu çıktı ses dizisinin kodunu çözmek için kullanır.

İlk üç modülün her birinin, çıkış sesini önceden tanımlanmış belirli bir sese göre koşullandırmak için koşullu hoparlör yerleştirmelerini destekleyebileceğini belirtmek gerekir.

Bark son derece kontrol edilebilir bir metin-konuşma modelidir, yani göreceğimiz gibi çeşitli ayarlarla kullanabilirsiniz.

Her şeyden önce, modeli ve işlemcisini yükleyin.

İşlemcinin buradaki rolü iki yönlüdür:

1. Giriş metnini tokenize etmek, yani modelin anlayabileceği küçük parçalara ayırmak için kullanılır. 
2. Konuşmacı gömülerini, yani üretimi koşullandırabilecek ses ön ayarlarını depolar.

In [9]:
from transformers import BarkModel, BarkProcessor

model = BarkModel.from_pretrained("suno/bark-small")
processor = BarkProcessor.from_pretrained("suno/bark-small")

config.json:   0%|          | 0.00/8.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/opt/conda/lib/python3.10/site-packages/transformers/models/encodec/modeling_encodec.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


generation_config.json:   0%|          | 0.00/4.91k [00:00<?, ?B/s]

speaker_embeddings_path.json:   0%|          | 0.00/61.1k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/353 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Bark çok yönlüdür ve işlemci aracılığıyla yüklenebilen bir hoparlör gömme kütüphanesi tarafından koşullandırılmış ses üretebilir.

In [10]:
inputs = processor("This is a test!", voice_preset="v2/en_speaker_3")
speech_output = model.generate(**inputs).cpu().numpy()

en_speaker_3_semantic_prompt.npy:   0%|          | 0.00/3.54k [00:00<?, ?B/s]

en_speaker_3_coarse_prompt.npy:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

en_speaker_3_fine_prompt.npy:   0%|          | 0.00/20.6k [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Ayrıca Fransızca ve Çince gibi kullanıma hazır çok dilli konuşmalar da oluşturabilir. Desteklenen dillerin bir listesini burada bulabilirsiniz. Aşağıda tartışılan MMS'in aksine, kullanılan dili belirtmek gerekli değildir, ancak giriş metnini ilgili dile uyarlamak yeterlidir.

In [11]:
inputs = processor("C'est un test!", voice_preset="v2/fr_speaker_1")
speech_output = model.generate(**inputs).cpu().numpy()

fr_speaker_1_semantic_prompt.npy:   0%|          | 0.00/2.62k [00:00<?, ?B/s]

fr_speaker_1_coarse_prompt.npy:   0%|          | 0.00/7.60k [00:00<?, ?B/s]

fr_speaker_1_fine_prompt.npy:   0%|          | 0.00/15.1k [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


Model ayrıca gülme, iç çekme ve ağlama gibi sözel olmayan iletişimler de oluşturabilir. Giriş metnini [boğazını temizler], [kahkaha] veya .... gibi ilgili ipuçlarıyla değiştirmeniz yeterlidir.

In [12]:
inputs = processor(
    "[clears throat] This is a test ... and I just took a long pause.",
    voice_preset="v2/fr_speaker_1",
)

speech_output = model.generate(**inputs).cpu().numpy()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


In [13]:
inputs = processor(
    "♪ In the mighty jungle, I'm trying to generate barks.",
)

speech_output = model.generate(**inputs).cpu().numpy()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


Tüm bu özelliklere ek olarak Bark, toplu işlemeyi destekler; bu, daha yoğun hesaplama pahasına aynı anda birkaç metin girişini işleyebileceğiniz anlamına gelir. GPU'lar gibi bazı donanımlarda, toplu işlem daha hızlı genel üretim sağlar, bu da örnekleri tek tek üretmektense hepsini bir kerede üretmenin daha hızlı olabileceği anlamına gelir.

In [14]:
input_list = [
    "[clears throat] Hello uh ..., my dog is cute [laughter]",
    "Let's try generating speech, with Bark, a text-to-speech model",
    "♪ In the jungle, the mighty jungle, the lion barks tonight ♪",
]

# also add a speaker embedding
inputs = processor(input_list, voice_preset="v2/en_speaker_3")
speech_output = model.generate(**inputs).cpu().numpy()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


In [15]:
sampling_rate = model.generation_config.sample_rate
Audio(speech_output[0], rate=sampling_rate)

In [16]:
Audio(speech_output[1], rate=sampling_rate)

In [17]:
Audio(speech_output[2], rate=sampling_rate)

## Massive Multilingual Speech (MMS)

Peki ya İngilizce dışında bir dilde önceden eğitilmiş bir model arıyorsanız? Massive Multilingual Speech (MMS), bir dizi konuşma görevini kapsayan başka bir modeldir, ancak **çok sayıda dili destekler**. Örneğin, 1.100'den fazla dilde konuşma sentezleyebilir.

Metinden konuşmaya yönelik MMS, son teknoloji TTS yaklaşımlarından biri olan VITS Kim ve diğerleri, 2021'e dayanmaktadır.

VITS, metni ham konuşma dalga biçimlerine dönüştüren bir konuşma üretim ağıdır. **Koşullu bir varyasyonel otomatik kodlayıcı gibi çalışır** ve giriş metninden ses özelliklerini tahmin eder. İlk olarak, spektrogramlar olarak temsil edilen akustik özellikler oluşturulur. Dalga biçimi daha sonra HiFi-GAN'dan uyarlanan dönüştürülmüş evrişimsel katmanlar kullanılarak çözülür. Çıkarım sırasında, metin kodlamaları yukarı örneklenir ve akış modülü ve HiFi-GAN kod çözücü kullanılarak dalga biçimlerine dönüştürülür. Bark'ta olduğu gibi, dalga biçimleri doğrudan oluşturulduğu için bir ses kodlayıcıya gerek yoktur.

MMS'i bir deneyelim ve İngilizce dışında bir dilde, örneğin Almanca, nasıl konuşma sentezleyebileceğimizi görelim. İlk olarak, doğru dil için model kontrol noktasını ve tokenizer'ı yükleyeceğiz:

In [18]:
from transformers import VitsModel, VitsTokenizer

model = VitsModel.from_pretrained("facebook/mms-tts-deu")
tokenizer = VitsTokenizer.from_pretrained("facebook/mms-tts-deu")

text_example = (
    "Ich bin Schnappi das kleine Krokodil, komm aus Ägypten das liegt direkt am Nil."
)

config.json:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/145M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/mms-tts-deu were not used when initializing VitsModel: ['flow.flows.0.wavenet.in_layers.0.weight_g', 'flow.flows.0.wavenet.in_layers.0.weight_v', 'flow.flows.0.wavenet.in_layers.1.weight_g', 'flow.flows.0.wavenet.in_layers.1.weight_v', 'flow.flows.0.wavenet.in_layers.2.weight_g', 'flow.flows.0.wavenet.in_layers.2.weight_v', 'flow.flows.0.wavenet.in_layers.3.weight_g', 'flow.flows.0.wavenet.in_layers.3.weight_v', 'flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'flow.flows.0.wavenet.res_skip_layers.0.weight_v', 'flow.flows.0.wavenet.res_skip_layers.1.weight_g', 'flow.flows.0.wavenet.res_skip_layers.1.weight_v', 'flow.flows.0.wavenet.res_skip_layers.2.weight_g', 'flow.flows.0.wavenet.res_skip_layers.2.weight_v', 'flow.flows.0.wavenet.res_skip_layers.3.weight_g', 'flow.flows.0.wavenet.res_skip_layers.3.weight_v', 'flow.flows.1.wavenet.in_layers.0.weight_g', 'flow.flows.1.wavenet.in_layers.0.weight_v', 'flow.flows.1.wavenet.in_layers.1.wei

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/496 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/47.0 [00:00<?, ?B/s]

In [19]:
import torch

inputs = tokenizer(text_example, return_tensors="pt")
input_ids = inputs["input_ids"]


with torch.no_grad():
    outputs = model(input_ids)

speech = outputs["waveform"]
Audio(speech, rate=16000)